In [57]:

#Import dependencies.

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
import requests
#1a import the Openweather API key
from configlast1 import weather_api_keys

In [58]:
# Create a set of random latitude and longitude combinations.
# Create a new set of 2,000 random latitudes and longitudes.

lats = np.random.uniform(low=-90.000, high=90.000, size =2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs


In [59]:
# Add the latitudes and longitudes to a list
coordinates = list(lats_lngs)

In [60]:
#for coordinate in lat_lngs:
    #print(coordinate[0],coordinate[1])

In [61]:
#Get the nearest city using the citipy module.
#1a Create a list for holding the cities
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#Print the city count to confirm sufficient count
len(cities)

705

In [50]:
#Perform an API call with the OpenWeatherMap
#Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys


In [118]:
city_urls = url + "&q=" + "aksu"
city_weatherss = requests.get(city_urls)
aksu_data = city_weatherss.json()
aksu_weather = aksu_data['weather'][0]['description']
aksu_weather



'clear sky'

In [114]:
city_data = []
# Create an empty list to hold the weather data.

# Create counters.
record_count = 1
set_count = 11

In [122]:
#Loop through all the cities in our list
#Latitude and longitude
#Maximum temperature
#Percent humidity
#Percent cloudiness
#Wind speed
#Weather description (for example, clouds, fog, light rain, clear sky)
for i, city in enumerate(cities):
    
    #Group cities in sets of 100 for logging purposes
    if (i % 50 == 0 and i >= 50):  
        set_count += 1
        record_count += 1  
        
    #Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Add 1 to the record count
    record_count += 1
    
    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #run an API request for each of the cities.
    try:
        #Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,"Weather Description": city_description})
    #If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass
#Indicate that Data loading is complete. 
#print("_______")
#print("Data Retrieval Complete")
#print("_______")

In [123]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df = city_data_df[['City', 'Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Weather Description']]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Lima,PE,-12.0432,-77.0282,59.40,86,90,13.80,NaN
1,Ushuaia,AR,-54.8000,-68.3000,38.86,75,75,25.32,NaN
2,Jamestown,US,42.0970,-79.2353,60.39,87,90,7.00,NaN
3,Vaini,TO,-21.2000,-175.2000,75.36,88,40,10.36,NaN
4,Pangnirtung,CA,66.1451,-65.7125,34.43,60,100,2.82,NaN
...,...,...,...,...,...,...,...,...,...
1295,Nabire,ID,-3.3667,135.4833,78.98,88,97,2.35,overcast clouds
1296,Praia Da Vitoria,PT,38.7333,-27.0667,71.85,88,90,7.92,overcast clouds
1297,Andenes,NO,69.3143,16.1194,40.98,87,39,6.91,scattered clouds
1298,Tual,ID,-5.6667,132.7500,81.00,81,95,17.40,overcast clouds


In [125]:
#create the output file (CSV)
output_data_file = "weather_data_challenge/WeatherPy_Database.csv"

#export to csv

city_data_df.to_csv(output_data_file, index_label='City_ID')
